# Day 7

In [311]:
var input = await System.IO.File.ReadAllLinesAsync("input.txt");

input

index,value
0,$ cd /
1,$ ls
2,150555 bch.lht
3,276291 ccqfdznj.sqg
4,dir csmqbhjv
5,dir czdqfr
6,dir fpfwfzrt
7,192660 qnbzgp
8,142026 rpphgdhp.jfr
9,dir sqphfslv


In [312]:
public interface IFileSystemObject
{
    string Name { get; }
}

public record Directory(string Name) : IFileSystemObject
{
    private Dictionary<string, IFileSystemObject> children = new Dictionary<string, IFileSystemObject>();

    public void Append(IFileSystemObject child) => children.Add(child.Name, child);

    public Directory GetDirectory(string name)
    {
        return children.TryGetValue(name, out var child) && child is Directory d
            ?  d
            : throw new InvalidOperationException($"Dirrectory {name} not found");
    }

    public int CalculateSize()
    {
        return children.Sum(c => c.Value switch
        {
            Directory d => d.CalculateSize(),
            File f => f.Size,
            _ => throw new ArgumentException()
        });
    }

    public IDictionary<Directory, int> CalculateSizeMap()
    {
        return children.Values
            .OfType<Directory>()
            .SelectMany(d => d.CalculateSizeMap())
            .Append(new KeyValuePair<Directory, int>(this, CalculateSize()))
            .ToDictionary(m => m.Key, m => m.Value);
    }
}

public record File(string Name, int Size) : IFileSystemObject;

In [313]:
var root = new Directory("/");
var path = new Stack<Directory>();

path.Push(root);

var current = root;

foreach (var instruction in input.Skip(1))
{
    if (instruction.StartsWith("dir"))
    {
        var name = instruction[4..];
        current.Append(new Directory(name));
    }
    else if (instruction.StartsWith("$ cd"))
    {
        var name = instruction[5..];

        if (name == "..")
        {
            current = path.Pop();
        }
        else
        {
            path.Push(current);
            current = current.GetDirectory(name);
        }
    }
    else if (instruction.StartsWith("$ ls"))
    {
    }
    else 
    {
        var i = instruction.IndexOf(" ");
        var size = Int32.Parse(instruction[..i]);
        var name = instruction[(i+1)..];

        current.Append(new File(name, size));
    }
}

root.CalculateSize()

47048086

In [314]:
var map = root.CalculateSizeMap();

map

key value Name brwncbh 238441 Name dtdzsqps 375884 Name czdqfr 8551 Name hrhqhcjg 8255 Name czdqfr 16806 Name fgmz 145921 Name cdqv 294789 Name cjtb 306568 Name llg 615499 Name lmgrr 306217 Name rcb 306217 Name fmmblb 1419115 Name czdqfr 1771159 Name rjnv 3048790 Name clmdlmc 627855 Name fmmblb 254906 Name czdqfr 269467 Name pgrtzw 32086 Name rwnqgjmm 507936 Name mnm 1037499 (169 more)

In [315]:
map.Where(m => m.Value <= 100000).Sum(m => m.Value)

1307902

In [316]:
const int total = 70000000;
const int required = 30000000;

var currentFree = total - root.CalculateSize();

map.OrderBy(m => m.Value).First(m => currentFree + m.Value >= required)


Key,Value
Directory Name: jswfprpl,7068748
